### First i install all the necessary libraries

In [ ]:
!pip install openai-whisper
!pip install yt_dlp
!pip install unsloth[all]

### Import all the packages here which are necessary 

In [ ]:
import os ,json,subprocess,uuid ,torch,whisper,glob,yt_dlp,io
from pathlib import Path
from IPython.display import Audio
from itertools import chain
import pandas as pd
import numpy as np 
from pydub import AudioSegment

### Step 1: Select the audio file from web or local or from anywhere ,
#### I  download the file from youtube this will dowload the file and save in your current directory

In [ ]:
Audio_dir = Path("/content/audio")
Audio_dir.mkdir(exist_ok=True)
url = ['https://www.youtube.com/watch?v=mKBbP4T5fbk&t=142s&ab_channel=SpeakEnglishWithTiffani']
ydl_opts = {
    'format': 'bestaudio/best',
    'extractaudio': True,
    'audioformat': 'm4a',
     'outtmpl': os.path.join(Audio_dir, '%(title)s.%(ext)s')
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download(url)

#### Here I check the file duration  using (Subprocess class) and using (ffprobe)

In [ ]:
def get_duration_sec(path:Path)->float:
  # return seconds using ffprob
  pr  = subprocess.run(['ffprobe','-v','error','-show_entries','format=duration','-of','json',str(path)],capture_output=True,text=True)
  try :
    return float(json.loads(pr.stdout)["format"]["duration"])
  except (json.JSONDecodeError,ValueError,KeyError) as e:
    print(f"Error getting duration for {path}:{e}")
    return 0.0

audio_paths = sorted(chain(
    Path('/content/audio').glob('*.m4a'),
    Path('/content/audio').glob('*.mp3'),
    Path('/content/audio').glob('*.wav'),
    Path('/content/audio').glob('*.webm'),
    ))
print(f" Found {len(audio_paths)} audio files: ")

total_duration = 0
for files in audio_paths:
  dur_sec = get_duration_sec(files)
  total_duration += dur_sec
  print(f" {files.name:<40} : {dur_sec/60:6.2f} min")

print(f"Total duration: {total_duration/60:6.2f} min")

### Step 2 : Cleaning the Voice using (ffmpeg and its some filters) and save in a directory 

In [ ]:
clean_Audio_dir = Path('/content/clean_audio')
clean_Audio_dir.mkdir(exist_ok=True)

def clean_audio(src:Path)->Path:
  #FFmpeg filters to clean and normalise audio
  out = clean_Audio_dir/f'{src.stem}_clean.wav'
  filters = (
      "highpass=f=90,"                # this is use to remove low frequency noise below 90hz
      "afftdn,"                       # this is FFT based denoiser to reduce broadband noise
      "loudnorm=I=-16:LRA=11:TP=-1.5," # Apply loudness normalization
      "dynaudnorm=f=200,"             #dynamic audio normalization to adjust volume based on given frame size
      "apad=pad_dur=0.1"             # add 100ms of silence padding at the end of the video
  )
  subprocess.run(["ffmpeg","-y","-i",str(src),
                  "-af",filters,"-ar","32000","-ac","1",str(out)],check=True)#sample rate 40000 and output as mono channel
  return out

In [ ]:
total_cleaned= []
total_duration_cleaned = 0
for files in audio_paths:
  clean_file = Path(clean_Audio_dir/f'{files.stem}_clean.wav')
  if clean_file.exists():
    print(f"Skipping {clean_file.name} file already Exist ")
    total_duration_cleaned += get_duration_sec(clean_file)
    total_cleaned.append(clean_file)
  else:
    print(f'Cleaning {files.name} - ',end="")
    total_duration_cleaned += get_duration_sec(files)
    total_cleaned.append(clean_audio(files))
    print("Done ",)

print(f"Total duration: {total_duration/60:6.2f} min")

### Step 3 : Now we are Transcribing(means convert the audio to text) the audio using whiper library 

In [ ]:
MODEL_SIZE = "large"  #@param ["tiny", "base", "small", "medium", "large"]
DEVICE = 'cuda'        #"cuda" tells PyTorch to use an NVIDIA GPU if available.
                      #"cpu" means run on the CPU (slower).
model = whisper.load_model(MODEL_SIZE,device=DEVICE)
Trancripts_dir = Path('/content/trancripts')
Trancripts_dir.mkdir(exist_ok=True)

for audio_path in total_cleaned:
  out_json = Trancripts_dir/f'{audio_path.stem}.json'

  if out_json.exists():
    print(f"Skipping {out_json.name} file already Exist ")
    continue

  print(f"Transcribing - {out_json.stem}..")

  result = model.transcribe(str(audio_path),word_timestamps=True,
                            fp16=(DEVICE=='cuda'),verbose=True)
  with out_json.open('w') as f:
    json.dump(result,f,indent=2)

  print(f"Saved {out_json.name} in Transcripts Directory ")
print("All transcried are Done")

### Step 4 : Now we are making the dataset using audio and transcribe file for train the model on our downloaded audio 

In [ ]:
def create_audio_text_dataset(json_file_path, audio_file_path, output_csv_path, audio_chunks_dir,chunk_duration=30):
    
    # Create output directory for audio segments
    os.makedirs(audio_chunks_dir, exist_ok=True)
    
    # Read the JSON file
    with open(json_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # Try to load audio processing libraries
  
    print("Using pydub for audio processing...")
    audio = AudioSegment.from_file(audio_file_path)
    sample_rate = audio.frame_rate
    print(f"Audio loaded successfully: {len(audio)/1000:.2f} seconds at {sample_rate} Hz")
    total_duration = len(audio) / 1000

    
    # Extract segments and create dataset rows
    segments = data.get('segments', [])
    
    rows = []
    chunk_id = 0
    current_chunk_start = 0.0
    current_chunk_texts = []
    current_chunk_end = 0.0
        
    for segment in segments:
        seg_start = segment.get('start', 0.0)
        seg_end = segment.get('end', 0.0)
        seg_text = segment.get('text', '').strip()
            
        if not seg_text:
                continue
            
        # Check if adding this segment would exceed 30 seconds
        potential_chunk_duration = seg_end - current_chunk_start
            
        # If this would make chunk too long (>30s), save current chunk and start new one
        if potential_chunk_duration > 30.0 and current_chunk_texts:
             # Save current chunk
            try:
                # Extract audio for current chunk
                start_ms = int(current_chunk_start * 1000)
                end_ms = int(current_chunk_end * 1000)
                    
                audio_chunk = audio[start_ms:end_ms]
                    
                # Save audio chunk to file
                audio_filename = f"chunks_{chunk_id:03d}.wav"
                audio_file_path = os.path.join(audio_chunks_dir, audio_filename)
                audio_chunk.export(audio_file_path, format="wav")
              
                combined_text = ' '.join(current_chunk_texts).strip()
                    
                # Create row with perfectly aligned audio and text
                audio_with_path = {
                    'path':  audio_file_path
                }
                    
                row = {
                    'audio': audio_with_path,
                    'text': combined_text,
                    'source': "0"
                }
                rows.append(row)
                    
                chunk_duration_actual = current_chunk_end - current_chunk_start
                print(f"Chunk {chunk_id}: {current_chunk_start:.1f}s-{current_chunk_end:.1f}s ({chunk_duration_actual:.1f}s), {len(combined_text)} chars")
                chunk_id += 1
                    
            except Exception as e:
                print(f"Error processing chunk {chunk_id}: {e}")
                
                # Start new chunk with current segment
            current_chunk_start = seg_start
            current_chunk_texts = [seg_text]
            current_chunk_end = seg_end
        else:
            # Add this segment to current chunk
            if not current_chunk_texts:  # First segment in chunk
                current_chunk_start = seg_start
            current_chunk_texts.append(seg_text)
            current_chunk_end = seg_end
        
        # Don't forget the last chunk
    if current_chunk_texts:
        try:
            start_ms = int(current_chunk_start * 1000)
            end_ms = int(current_chunk_end * 1000)
                
            audio_chunk = audio[start_ms:end_ms]
                
            # Save audio chunk to file
            audio_filename = f"chunks_{chunk_id:03d}.wav"
            audio_file_path = os.path.join(audio_chunks_dir, audio_filename)
            audio_chunk.export(audio_file_path, format="wav")
                
            combined_text = ' '.join(current_chunk_texts).strip()
                
            audio_with_path = {
                'path': audio_file_path
            }
                
            row = {
                'audio': audio_with_path,
                'text': combined_text,
                'source': "0"
            }
            rows.append(row)
                
            chunk_duration_actual = current_chunk_end - current_chunk_start
            print(f"Chunk {chunk_id}: {current_chunk_start:.1f}s-{current_chunk_end:.1f}s ({chunk_duration_actual:.1f}s), {len(combined_text)} chars")
                
        except Exception as e:
            print(f"Error processing final chunk: {e}")
    # Create DataFrame and save to CSV
    df = pd.DataFrame(rows)
    df.to_json(output_csv_path, index=False,orient="records")
    
    print(f"\n{'='*60}")
    print(f"DATASET CREATED SUCCESSFULLY!")
    print(f"{'='*60}")
    print(f" Dataset file: {output_csv_path}")
    print(f" Audio segments folder: {audio_chunks_dir}")
    print(df.head().to_string())
    return df

In [ ]:
json_file = r"/content/trancripts/ENGLISH MASTERCLASS ｜ THINK & SPEAK ENGLISH [FULL LESSON]_clean.json"
audio_file = r"/content/clean_audio/ENGLISH MASTERCLASS ｜ THINK & SPEAK ENGLISH [FULL LESSON]_clean.wav"
dataset_json = r"/content/our_dataset/dataset_json.json"
audio_segments_dir = r"/content/myaudio_segments"
    
our_dataset = Path("/content/our_dataset")
our_dataset.mkdir(exist_ok = True)
# Check if files exist
if not os.path.exists(json_file):
    print(f"JSON file not found: {json_file}")
    exit(1)
    
if not os.path.exists(audio_file):
    print(f"Audio file not found: {audio_file}")
    print("Please ensure the audio file is in the Downloads folder")
    exit(1)
    
print("Starting dataset creation...")
main_df = create_audio_text_dataset(json_file, audio_file,dataset_json, audio_segments_dir)

In [ ]:
main_df

### Step 5 : Gathering the unsloth/csm-1b model from hugging face

In [ ]:
# Gatting the predefined model of unsloth 
from unsloth import FastModel
from transformers import CsmForConditionalGeneration

model_name = "unsloth/csm-1b"

model,processor = FastModel.from_pretrained(
    model_name = model_name,
    max_seq_length = 2048,
    dtype = None,
    auto_model = CsmForConditionalGeneration,
    load_in_4bit = False,
    full_finetuning = False)

In [ ]:
print(model)

In [ ]:
model  = FastModel.get_peft_model(
     model,
     r= 32,
     target_modules = ["q_proj","k_proj","v_proj",
                       "o_proj","gate_proj","up_proj",
                       "down_proj",],
     lora_alpha = 16,
     lora_dropout = 0,
     bias = "none",
     use_gradient_checkpointing = "unsloth",
     randomstate = 3407,
     use_rslora = True,
     loftq_config = None,
 )

In [ ]:
import time
run_name =model_name.split("/")[-1]+'-lora-ft'+time.strftime("_%Y%m%d_%H%M%S")
print(run_name)


In [ ]:
model.print_trainable_parameters()

### Step 6: Get the dataset which you made for split the data into train and test dataset 

In [ ]:
sample_rate = 24000

import os
from transformers import AutoProcessor
from datasets import load_dataset,Audio

processor = AutoProcessor.from_pretrained("unsloth/csm-1b")
# raw_ds = load_dataset(exa_dataset,split="train")

dataset = load_dataset("json",data_files="/content/our_dataset/dataset_json.json")
dataset = dataset.cast_column("audio", Audio())
raw_ds = dataset["train"]

print(f'Dataset loaded with features: {raw_ds.features["audio"]}')

if "source" not in raw_ds.column_names:
   print("Unsloth No speaker Found")
   new_column = ["0"]*len(raw_ds)
   raw_ds = raw_ds.add_column(name="source",column=new_column)
elif "source" in raw_ds.column_names:
   speaker_key = "source"

target_sampling_rate = sample_rate
raw_ds = raw_ds.cast_column("audio",Audio(sampling_rate=target_sampling_rate))

import math
from datasets import DatasetDict

totalrows = len(raw_ds)
test_rows = min(30,max(1,math.ceil(0.10*totalrows)))

split:DatasetDict = raw_ds.train_test_split(test_size=test_rows,shuffle=True,seed=42)

rawTrainDs = split["train"]
rawTestDs = split["test"]

print(f"Train rows: {len(rawTrainDs)}")
print(f"Eval rows: {len(rawTestDs)}")

In [ ]:
max_audio_length = 0
max_text_length = 0 # Initialize max_text_length
for row in raw_ds:
    text_length = len(row['text'])
    audio_length = len(row['audio']['array'])
    max_text_length = max(max_text_length, text_length)
    max_audio_length = max(max_audio_length, audio_length)

print(f"Maximum text length: {max_text_length}")
print(f"Maximum audio length: {max_audio_length}")

### Step 7 :  Now prepocess the data or cleaning the dataset and structure the dataset to understand by the model

In [ ]:
def preprocess(example):
  conversation = [
      {
          "role": str(example["source"]),
          "content": [
              {"type": "text", "text": example["text"]},
              {"type": "audio", "audio": example["audio"]["array"]} # Pass the numpy array
          ],
      }
  ]
  try :
    model_inputs = processor.apply_chat_template(
        conversation,
        tokenize=True,
        return_dict =True,
        output_labels = True,
        text_kwargs={
            "padding":"max_length",
            "max_length":max_text_length,
            "pad_to_multiple_of":8,
            "padding_side":"right",
        },
        audio_kwargs={
            "sampling_rate":24000,
            "padding":"max_length",
            "max_length":max_audio_length,
        },
        common_kwargs = {"return_tensors":"pt"}
    )
  except Exception as e:
    print(e)
    return None
  required_keys = ["input_ids","attention_mask","labels","input_values","input_values_cutoffs"] # Corrected typo here
  processed = {}

  for key in required_keys:
    if key not in model_inputs:
      print(f"{key} missing from model_inputs")
      return None
    value = model_inputs[key][0]
    processed[key] = value

  if not all(isinstance(processed[key],torch.Tensor) for key in processed):
    print("Not all values in processed are tensors")
    return None
  return processed

processed_train_ds = rawTrainDs.map(preprocess,remove_columns=rawTrainDs.column_names,desc="Preprocessing Train data")
processed_test_ds = rawTestDs.map(preprocess,remove_columns=rawTestDs.column_names,desc="Preprocessing Test data")

In [ ]:
print(processed_train_ds)

### Step 8: We can train the model 

In [ ]:
from transformers import TrainingArguments,Trainer
from unsloth import is_bfloat16_supported

trainer=Trainer(
    model =model,
    train_dataset = processed_train_ds,
    eval_dataset = processed_test_ds,
    args = TrainingArguments(
        per_device_train_batch_size =1,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs =3,
        eval_strategy = "steps",
        eval_steps = 0.2,
        learning_rate = 2e-4, #for lora,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps =1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "constant",
        seed = 3407,
        output_dir = "outputs",
        report_to = 'tensorboard',
        logging_dir = f"logs/{run_name}"
        )
)

In [ ]:
trainer_stats = trainer.train()

### Step 9 : Now we Do Fine-tuning to check the performance of the model

In [ ]:
import soundfile as sf
from IPython.display import Audio, display  
text = """Resemble AI can clone a voice with just a few minutes of recorded data, making it possible to 
generate realistic speech that closely matches the original speaker’s tone, style, and personality
.Learning a new skill is like planting a seed; with patience, effort, and care, it will grow into something meaningful and rewarding over time.
In today’s fast-paced digital world, communication has become more instant than ever, but the value of genuine human connection remains timeless.
Technology is changing the way we live, work, and communicate with each other.
Artificial intelligence is not here to replace humans, but to help us work smarter and faster.
Reading books can transport you into different worlds and expand your imagination.
"""
speaker_id = 0
inputs = processor(
    f"[{speaker_id}]{text}",
    add_special_tokens = True,
    return_tensors = "pt",
).to("cuda")

audio_values = model.generate(
    **inputs,
    max_new_tokens=375,#125 tokens is 10 seconds of audio ,for longer speech increase this 
    #play with below parameters for better output
    depth_decoder_temperature = 0.6,
    depth_decoder_top_k = 0,
    depth_decoder_top_p = 0.9,
    temperature = 0.8,
    top_k = 50,
    top_p = 1.0,
    output_audio = True
)

audio = audio_values[0].to(torch.float32).cpu().numpy()
sf.write("finetuned.wav",audio ,sample_rate)
print("Fine-tuned:")
display(Audio(audio, rate=sample_rate))
print("Real: ")
display(Audio(rawTestDs[3]["audio"]["array"],rate =sample_rate))

### Step 10 : Now we Clone the audio

In [ ]:
speaker_id = 0

cloned = rawTestDs[1]["audio"]["array"] 
cloned_text = rawTestDs[1]["text"]
conversation = [
{"role": str(speaker_id),"content": [{"type": "text", "text": cloned_text},{"type": "audio", "audio": cloned}],}
  ,{"role": str(speaker_id),"content": [{"type": "text", "text": text}],}
                 ]
inputs = processor.apply_chat_template(
        conversation,
        tokenize=True,
        return_dict =True,
        return_tensors='pt')

inputs = {k: v.to("cuda") for k, v in inputs.items()}

audio_values = model.generate(
    **inputs,
    max_new_tokens=375,#125 tokens is 10 seconds of audio ,for longer speech increase this ~12.5 tokens ≈ 1 second of audio
    #play with below parameters for better output
    depth_decoder_temperature = 0.6,
    depth_decoder_top_k = 0,
    depth_decoder_top_p = 0.9,
    temperature = 0.8,
    top_k = 50,
    top_p = 1.0,
    output_audio = True
)

audio = audio_values[0].to(torch.float32).cpu().numpy()
sf.write("clonedVoice.wav",audio ,sample_rate)
print("cloned")
display(Audio(audio, rate=sample_rate))
print("Real: ")
display(Audio(rawTestDs[3]["audio"]["array"],rate =sample_rate))